In [1]:
!pip install mxnet
!pip install tqdm
!pip install sentencepiece
!pip install --upgrade mxnet gluonnlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 17.0 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.5/344.5 kB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp310-cp310-linux_x86_64.whl size=645679 sha256=7a92d182a2f681dcf3b34252d0bd773c3274ece88cb2e78e6959ff87e3a6af2f
  Stored in directory: /root/.cache/pip/wheels/1a/1e/0d/99f55911d90f2b95b9f7c176d5813ef3622894a4b30fde6bd3
Successfully built gluonnlp


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
##GPU 사용
import torch
device = torch.device("cuda:0")

Load model and tokenizer

In [4]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 62.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.9 MB/s eta 0:00:00


In [5]:
from transformers import ElectraTokenizer
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")

In [6]:
from transformers import ElectraForSequenceClassification
model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-base-v3-discriminator")

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: ['classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
model.cuda()

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(35000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): L

Tokenizer example

In [8]:
tokenizer.tokenize("[CLS] 한국어 ELECTRA를 공유합니다. [SEP]")

['[CLS]', '한국어', 'EL', '##EC', '##TRA', '##를', '공유', '##합니다', '.', '[SEP]']

In [9]:
tokenizer.convert_tokens_to_ids(['[CLS]', '한국어', 'EL', '##EC', '##TRA', '##를', '공유', '##합니다', '.', '[SEP]'])

[2, 11229, 29173, 13352, 25541, 4110, 7824, 17788, 18, 3]

Dataset load

In [11]:
import pandas as pd
train = pd.read_csv('/../corpus/train_sentence_final.tsv', sep='\t')
valid = pd.read_csv('/../corpus/valid_sentence_final.tsv', sep='\t')
test = pd.read_csv('/../corpus/test_sentence_final.tsv', sep='\t')

print(train.shape)
print(valid.shape)
print(test.shape)

(30576, 2)
(7644, 2)
(9555, 2)


In [12]:
train.head(10)

,sentence,label
0,학교 선생님들은 일기를 통해 학생의 일상을 엿보고자 한다.,1
1,그렇게 둘이 안 헤어지고 결혼을 하게 되어서 기쁘다.,1
2,그런데 화장실에 가려는데 어제 아침에 변기가 막힌 것이 생각났다.,1
3,그러나 그날은 달랐다.,1
4,어떻게 생활 하셨나?,1
5,약속을 잡고도 나가기를 귀찮아했었는데 밖에서 시간을 보내보자는 생각도 들었다.,1
6,누군가에게 마음을 내어 속풀이를 하는 것도 그 나름대로 상대방을 안심시키는 방법이 ...,1
7,파도풀도 생각보다 재미있었다.,0
8,동그란 이마에 손을 갖다 댔다.,1
9,6대6으로 경기를 했다.,0


In [13]:
sentences = ["[CLS] " + str(sen) + " [SEP]" for sen in train['sentence']]
sentences[:10]

['[CLS] 학교 선생님들은 일기를 통해 학생의 일상을 엿보고자 한다. [SEP]',
 '[CLS] 그렇게 둘이 안 헤어지고 결혼을 하게 되어서 기쁘다. [SEP]',
 '[CLS] 그런데 화장실에 가려는데 어제 아침에 변기가 막힌 것이 생각났다. [SEP]',
 '[CLS] 그러나 그날은 달랐다. [SEP]',
 '[CLS] 어떻게 생활 하셨나? [SEP]',
 '[CLS] 약속을 잡고도 나가기를 귀찮아했었는데 밖에서 시간을 보내보자는 생각도 들었다. [SEP]',
 '[CLS] 누군가에게 마음을 내어 속풀이를 하는 것도 그 나름대로 상대방을 안심시키는 방법이 될 수 있으리라 생각한다. [SEP]',
 '[CLS] 파도풀도 생각보다 재미있었다. [SEP]',
 '[CLS] 동그란 이마에 손을 갖다 댔다. [SEP]',
 '[CLS] 6대6으로 경기를 했다. [SEP]']

In [14]:
val_sentences = ["[CLS] " + str(sen) + " [SEP]" for sen in valid['sentence']]
test_sentences = ["[CLS] " + str(sen) + " [SEP]" for sen in test['sentence']]
val_sentences[:10]

['[CLS] 처음에 꿈 꾸었던 설렘과 감동을 주자는 내 목표는 어느새 벽이 되어 있다. [SEP]',
 '[CLS] 표지판에 있는 새우는 하얀 색이었는데 보지 못해서 아쉬웠다.  [SEP]',
 '[CLS] 오늘 세종시에 와서 외숙모와 삼촌과 놀았다. [SEP]',
 '[CLS] 2020년 쥐띠의 해다. [SEP]',
 '[CLS] 오늘 다시 홈페이지보니까 만 19세 이상부터 만59세인가(?)까지는 못해서 놀랐다. [SEP]',
 '[CLS] 그러나, 할아버지의 손을 잡고 있는 건 나의 손이 아닌 지팡이였고 나는 할아버지와의 조금의 간격을 둔 채 뒤따라  걷기 시작했다. [SEP]',
 '[CLS] 사전에서는 자존감을 자신이 사랑받을 만한 가치가 있는 소중한 존재이고 어떤 서오가를 이루어낼 만한 유능한 사람이라고 믿는 마음이라고 정의한다. [SEP]',
 '[CLS] 나는 그 사람의 얼굴을 보고 첫눈에 반했다 [SEP]',
 '[CLS] 엄마는 Lostin the Woods가 좋다고했다. [SEP]',
 '[CLS] 친구와 단둘이서 여행을 가게 되어 들떠 있던 나에게 어머니께서는 자신이 암이라는 말을 꺼내지 못하셨다고 한다. [SEP]']

In [15]:
# labels
labels = train['label'].values
labels

array([1, 1, 1, ..., 0, 0, 1])

In [16]:
val_labels = valid['label'].values
test_labels = test['label'].values

In [17]:
# tokenizing
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

print(sentences[0])
print(tokenized_texts[0])

[CLS] 학교 선생님들은 일기를 통해 학생의 일상을 엿보고자 한다. [SEP]
['[CLS]', '학교', '선생', '##님', '##들', '##은', '일기', '##를', '통해', '학생', '##의', '일상', '##을', '엿보', '##고', '##자', '한다', '.', '[SEP]']


In [18]:
val_tokenized_texts = [tokenizer.tokenize(sent) for sent in val_sentences]
test_tokenized_texts = [tokenizer.tokenize(sent) for sent in test_sentences]

Setting parameters

In [19]:
MAX_LEN = 64
NUM_EPOCHS = 5
BATCH_SIZE = 32
learning_rate =  5e-5
warmup_ratio = 0.1
max_grad_norm = 1
log_interval = 200

Convert tokens to index

In [20]:
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
val_input_ids = [tokenizer.convert_tokens_to_ids(x) for x in val_tokenized_texts]
test_input_ids = [tokenizer.convert_tokens_to_ids(x) for x in test_tokenized_texts]

Padding

In [22]:
from keras.utils import pad_sequences
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype='long', truncating='post', padding='post')
val_input_ids = pad_sequences(val_input_ids, maxlen=MAX_LEN, dtype='long', truncating='post', padding='post')
test_input_ids = pad_sequences(test_input_ids, maxlen=MAX_LEN, dtype='long', truncating='post', padding='post')

input_ids[0]

array([    2,  6351,  7015,  4660,  4006,  4112, 10997,  4110,  6260,
        6409,  4234,  7674,  4292, 21638,  4219,  4195,  6217,    18,
           3,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0])

Attention mask

In [23]:
attention_masks = []
for seq in input_ids :
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask)
print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [24]:
val_attention_masks = []
for seq in val_input_ids :
  seq_mask = [float(i>0) for i in seq]
  val_attention_masks.append(seq_mask)

test_attention_masks = []
for seq in test_input_ids :
  seq_mask = [float(i>0) for i in seq]
  test_attention_masks.append(seq_mask)

convert a NumPy ndarray to a PyTorch Tensor

In [25]:
train_inputs = torch.tensor(input_ids)
valid_inputs = torch.tensor(val_input_ids)
test_inputs = torch.tensor(test_input_ids)

train_labels = torch.tensor(labels)
valid_labels = torch.tensor(val_labels)
test_labels = torch.tensor(test_labels)

train_masks = torch.tensor(attention_masks)
valid_masks = torch.tensor(val_attention_masks)
test_masks = torch.tensor(test_attention_masks)

Random Sampling and DataLoader

In [27]:
from torch.utils.data import Dataset, TensorDataset, DataLoader, RandomSampler, SequentialSampler
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler = train_sampler, batch_size = BATCH_SIZE)

valid_data = TensorDataset(valid_inputs, valid_masks, valid_labels)
valid_sampler = RandomSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler = valid_sampler, batch_size = BATCH_SIZE)

test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler = test_sampler, batch_size = BATCH_SIZE)

In [28]:
# Optimizer & Scheduler
from transformers import AdamW, get_linear_schedule_with_warmup
optimizer = AdamW(model.parameters(),
                  lr =learning_rate,
                  eps = 1e-8
                  )
epochs = NUM_EPOCHS
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [29]:
import datetime
import numpy as np
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

Training Model

In [30]:
import random, time
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
model.zero_grad()
for epoch_i in range(0, epochs):
    # ========================================
    #               Training
    # ========================================
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    t0 = time.time() # starting time
    total_loss = 0

    # train mode
    model.train()

    for step, batch in enumerate(train_dataloader):
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch

        # Forward
        outputs = model(b_input_ids,
                        token_type_ids=None,
                        attention_mask=b_input_mask,
                        labels=b_labels)

        # loss
        loss = outputs[0]
        total_loss += loss.item()

        # Backward
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # update parameters
        optimizer.step()
        scheduler.step()
        model.zero_grad()

    avg_train_loss = total_loss / len(train_dataloader)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))

    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    t0 = time.time() # starting time
    model.eval()
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    for batch in valid_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        with torch.no_grad():
            # Forward 수행
            outputs = model(b_input_ids,
                            token_type_ids=None,
                            attention_mask=b_input_mask)

        logits = outputs[0]

        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # accuracy
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 5 ========
Training...
  Batch   500  of    956.    Elapsed: 0:02:48.

  Average training loss: 0.38
  Training epcoh took: 0:05:22

Running Validation...
  Accuracy: 0.87
  Validation took: 0:00:28

======== Epoch 2 / 5 ========
Training...
  Batch   500  of    956.    Elapsed: 0:02:49.

  Average training loss: 0.21
  Training epcoh took: 0:05:23

Running Validation...
  Accuracy: 0.86
  Validation took: 0:00:29

======== Epoch 3 / 5 ========
Training...
  Batch   500  of    956.    Elapsed: 0:02:49.

  Average training loss: 0.11
  Training epcoh took: 0:05:23

Running Validation...
  Accuracy: 0.87
  Validation took: 0:00:29

======== Epoch 4 / 5 ========
Training...
  Batch   500  of    956.    Elapsed: 0:02:49.

  Average training loss: 0.06
  Training epcoh took: 0:05:22

Running Validation...
  Accuracy: 0.87
  Validation took: 0:00:28

======== Epoch 5 / 5 ========
Training...
  Batch   500  of    956.    Elapsed: 0:02:49.

  Average training loss: 0.03
  T

Test Data

In [31]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

def get_clf_eval(y_test, pred) :
  print('y_test', y_test)
  print('pred', pred)
  confusion = confusion_matrix(y_test, pred)
  accuracy = accuracy_score(y_test, pred)
  precision = precision_score(y_test, pred)
  recall = recall_score(y_test, pred)
  print('오차행렬')
  print(confusion)
  print('정확도:{0:.4f}, 정밀도:{1:.4f}, 재현율:{2:.4f}'.format(accuracy,precision,recall))


def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten() # 예상한거
    y_hat.append(pred_flat)
    labels_flat = labels.flatten() # 진짜 label
    real_y.append(labels_flat)
    # print("pred_flat", pred_flat)
    # print("labels_flat", labels_flat)
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [32]:
t0 = time.time()
model.eval()
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

y_hat = []
real_y = []

for step, batch in enumerate(test_dataloader):
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch

    with torch.no_grad():
        outputs = model(b_input_ids,
                        token_type_ids=None,
                        attention_mask=b_input_mask)
    logits = outputs[0]
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    # accuracy
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

print("")
print("Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
print("Test took: {:}".format(format_time(time.time() - t0)))

  Batch   100  of    299.    Elapsed: 0:00:11.
  Batch   200  of    299.    Elapsed: 0:00:22.

Accuracy: 0.87
Test took: 0:00:34
